In [15]:
# A standard game of checkers. 8x8 board.
class Checkers():
    def __init__(self):
        self.initial = Checkers_Board()

    def remove_self_intersections(self, i, board):
        # There are 4 possible moves.
        turn = board.to_move
        if(turn == 'W'):
            piece = board.P[turn][i]
            moves = []
            if(piece[1] < 7):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            if(board.Kings[turn][i] and piece[1] > 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            available_moves = []
            for m in moves:
                if m in board.P['W']:
                    continue
                available_moves.append(m)
            return available_moves
            
        if(turn == 'B'):
            piece = board.P[turn][i]
            moves = []
            if(piece[1] > 0 ):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            if(board.Kings[turn][i] and piece[1] < 8):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            available_moves = []
            for m in moves:
                if m in board.P['B']:
                    continue
                available_moves.append(m)
            return available_moves

    def capture_cycle(self, position, captured, board):
        directions = []
        p = (position[0] - 1, position[1] - 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] + 1, position[1] - 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] - 1, position[1] + 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        p = (position[0] + 1, position[1] + 1)
        if p[0] in range(8) and p[1] in range(8): directions.append(p)
        available_directions = []
        for d in directions:
            # we don't want to recapture and we can not do anything
            # if one of our pieces is  in the way. 
            if d not in captured and d not in board.P[board.to_move]:
                available_directions.append(d)
        capture_directions = 0
        for d in available_directions:
            rival = 'B' if board.to_move == 'W' else 'W'
            if d in board.P[rival]:
                if d in captured: continue
                # we have a rival piece, now we need to see if we can jump it.
                direction = (d[0] - position[0], d[1] - position[1])
                destination = (d[0] + direction[0], d[1] + direction[1])
                if destination[0] not in range(8) or destination[1] not in range(8): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in board.P['W'] or destination in board.P['B']: continue
                # At this point we have a valid square to land on.
                # We need to add the piece to captured and call capture_cycle again
                captured.append(d)
                capture_directions += 1
                board.capture_cycle(destination, captured)
        if(len(captured) > 0):
            captured.append(None)
            captured.append(position)
            captured.append(None)
                 
    
    def generate_capture_actions(self, i, piece, moves, board):
        captured = []
        other_player = 'B' if board.to_move == 'W' else 'W'
        other_player_piece_locations = board.P[other_player]
        best_capture = None
        for move in moves:
            # can we take a piece?
            if move in other_player_piece_locations:
                # We need to check whether the destination is valid
                direction = (move[0] - piece[0], move[1] - piece[1])
                destination = (move[0] + direction[0], move[1] + direction[1])
                if destination[0] not in range(8) or destination[1] not in range(8): continue
                # We also need to check whether the destination is open. That is, clear.
                if destination in board.P['W'] or destination in board.P['B']: continue
                # At this point we have a valid square to land on.
                captures = [move]
                final_destination = None
                self.capture_cycle(destination, captures, board)
                captures = list(dict.fromkeys(captures))
                if best_capture == None:
                    best_capture = captures
                else:
                    if(len(captures) > len(best_capture)):
                        best_capture = captures
        return [i, best_capture]

    # action is a tuple:
    # (i, [taken_pieces, None, final_position], make_king)
    def actions(self, board):
        actions = []
        for i in range(board.piece_count):
            turn = board.to_move
            piece = board.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(i, board)
            # If there are no moves ,just continue.
            if(len(moves) == 0): continue
            action = self.generate_capture_actions(i, piece, moves, board)
            actions.append(action)
        capture_only_actions = []
        for a in actions:
            if a[1] != None:
                capture_only_actions.append(a)
        if len(capture_only_actions) > 0:
            return capture_only_actions

        # if capture_only_actions is of size zero, 
        # we need to look into just moving pieces
        actions = []
        for i in range(board.piece_count):
            turn = board.to_move
            piece = board.P[turn][i]
            # If the piece is taken, just continue.
            if piece == None: continue
            moves = self.remove_self_intersections(i, board)
            for move in moves:
                actions.append([i, [None, move]])
        return actions

    def promote_king(self, piece_index, board):
        piece = board.P[board.to_move][piece_index]
        if board.to_move == 'W':
            if piece[1] == 7:
                board.Kings['W'][piece_index] = True
        if board.to_move == 'B':
            if piece[1] == 0:
                board.Kings['B'][piece_index] = True       
    
    def result(self, action, board):
        captured = None
        piece_index = action[0]
        move_data = action[1]
        k = move_data.index(None)
        board.P[board.to_move][piece_index] = move_data[k+1]
        # check if promotion is needed
        self.promote_king(piece_index, board)
        # if there is no captures, exit
        if k == 0:
            return
        # else we have to register captures
        captured = move_data[:k]
        other_player = 'B' if board.to_move == 'W' else 'W'
        for c in captured:
            c_index = board.P[other_player].index(c)
            #self.P[other_player][c_index] = None

class Checkers_Board():
    def __init__(self):
        # initial positions
        W = [(x,y) for y in range(0,3) for x in range(y%2,8,2)]
        B = [(x,y) for y in range(5,8) for x in range(y%2,8,2)]
        # for debugging
        B[0] = (3,3)
        B[1] = (5,3)
        self.piece_count = 12
        self.P = {'W' : W, 'B' : B}
        # we record king status as a mapping from location to boolean
        W_King = [False for i in range(self.piece_count)]
        B_King = [False for i in range(self.piece_count)]
        self.Kings = {'W' : W_King,'B' : B_King}
        self.to_move = 'W'
    def noe

In [18]:
c = Checkers()
cb = Checkers_Board()
actions = c.actions(cb)
for a in actions:
    print(a)
    c.result(a, cb)

[9, [(3, 3), None, (4, 4)]]
[10, [(3, 3), None, (2, 4)]]
[11, [(5, 3), None, (4, 4)]]
